# ENTRENAMIENTO DE DATOS

### LIBRERIAS

In [2]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

### CARGAR DATOS PROCESADOS

In [3]:
df = pd.read_csv("../data/processed/dataset_clean.csv")

### Definir características y variable objetivo

In [4]:
features = ['duration_min', 'explicit', 'danceability', 'energy', 'key', 
            'loudness', 'mode', 'speechiness', 'acousticness', 
            'instrumentalness', 'liveness', 'valence', 'tempo','track_genre_encoded']

X = df[features]
y = df['popularity']

### Normalización de datos

In [5]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

### División en entrenamiento y prueba

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

### ENTRENAMIENTO Y GUARDADO DE MODELOS

In [7]:
modelos = {
    "Regresión Lineal": LinearRegression(),
    "Random Forest": RandomForestRegressor(random_state=42),
    "Gradient Boosting": GradientBoostingRegressor(random_state=42),
    "XGBoost": XGBRegressor(random_state=42),
    "SVR": SVR()
}

### Carpeta para guardar modelos

In [8]:
models_dir = "../models/"
os.makedirs(models_dir, exist_ok=True)

### Evaluar cada modelo y guardar sus resultados

In [9]:
mejor_modelo = None
mejor_rmse = float("inf")

print("\nEntrenando modelos...")
for nombre, modelo in modelos.items():
    modelo.fit(X_train, y_train)
    predicciones = modelo.predict(X_test)

    rmse = np.sqrt(mean_squared_error(y_test, predicciones))
    mae = mean_absolute_error(y_test, predicciones)
    r2 = r2_score(y_test, predicciones)

    print(nombre, "- RMSE:", round(rmse, 1), "- MAE:", round(mae, 1), "- R²:", round(r2, 3))

    # Guardar modelo en la carpeta "models"
    modelo_path = os.path.join(models_dir, f"{nombre.replace(' ', '_')}.pkl")
    joblib.dump(modelo, modelo_path)

    # Actualizar el mejor modelo según RMSE
    if rmse < mejor_rmse:
        mejor_rmse = rmse
        mejor_modelo = modelo_path

print("\nMejor modelo seleccionado:", mejor_modelo)


Entrenando modelos...
Regresión Lineal - RMSE: 22.1 - MAE: 18.4 - R²: 0.028
Random Forest - RMSE: 15.9 - MAE: 11.0 - R²: 0.498
Gradient Boosting - RMSE: 20.3 - MAE: 16.2 - R²: 0.182
XGBoost - RMSE: 17.4 - MAE: 12.9 - R²: 0.396
SVR - RMSE: 21.3 - MAE: 16.9 - R²: 0.098

Mejor modelo seleccionado: ../models/Random_Forest.pkl


### OPTIMIZACIÓN DEL MEJOR MODELO

In [10]:
param_dist = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2'],
    'bootstrap': [True, False],
    'criterion': ['squared_error', 'absolute_error']
}

### Cargar el mejor modelo encontrado

In [11]:
modelo_final = joblib.load(mejor_modelo)

### Optimizar el modelo con RandomizedSearchCV

In [ ]:
search = RandomizedSearchCV(modelo_final, param_dist, n_iter=3, cv=5, scoring='neg_mean_squared_error', n_jobs=1, random_state=42, verbose=2)
search.fit(X_train, y_train)

print("\nMejor modelo optimizado:", search.best_params_)
print("Error optimizado (RMSE):", round(np.sqrt(-search.best_score_), 1))

Fitting 5 folds for each of 3 candidates, totalling 15 fits


### Guardar el modelo final

In [ ]:
modelo_final_path = os.path.join(models_dir, "modelo_final.pkl")
joblib.dump(search.best_estimator_, modelo_final_path)
print("Modelo final guardado en:", modelo_final_path)
